In [1]:
example_input_data = [
    {
        "CreditScore": 502,
        "Geography": "France",
        "Gender": "Female",
        "Age": 42,
        "Tenure": 8,
        "Balance": 159660.8,
        "NumOfProducts": 3,
        "HasCrCard": 1,
        "IsActiveMember": 0,
        "EstimatedSalary": 113931.57
    },
    {
        "CreditScore": 699,
        "Geography": "France",
        "Gender": "Female",
        "Age": 39,
        "Tenure": 1,
        "Balance": 0.0,
        "NumOfProducts": 2,
        "HasCrCard": 0,
        "IsActiveMember": 0,
        "EstimatedSalary": 93826.63
    }
]

In [20]:
import bentoml
import torch
import logging
import pandas as pd # <-- NEW: Needed for DataFrame operations
import numpy as np  # <-- NEW: Needed for array operations
import torch.nn as nn # <-- NEW: Ensure base classes are available if needed

logging.basicConfig(level=logging.INFO)

# --- MODEL ARTIFACT TAGS ---
# 🚨 NOTE: Ensure these tags match your bentoml models list output
# If you didn't tag with 'latest', replace with the specific tags (e.g., :tvyz5egkl6swwqqb)
model_tag = "churn_prediction_model:latest" #tvyz5egkl6swwqqb" 
preprocessor_tag = "churn_preprocessor:latest" #t6wrgygkl6swwqqb"

# --- 1. Load the Model ---
print(f"Loading Model: {model_tag}")
try:
    # 1. Get the BentoModel object
    bento_model = bentoml.models.get(model_tag)
    # 2. Load the PyTorch model with the weights_only fix
    model = bentoml.pytorch.load_model(bento_model, weights_only=False)
    model.eval() # Set model to evaluation mode
    print("✅ PyTorch Model loaded successfully.")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    model = None

# --- 2. Load the Preprocessor (SKLearn) ---
print(f"\nLoading Preprocessor: {preprocessor_tag}")
try:
    # 1. Get the BentoModel object
    bento_preprocessor = bentoml.models.get(preprocessor_tag)
    # 2. Load the preprocessor (assumed to be Scikit-Learn based)
    preprocessor = bentoml.sklearn.load_model(bento_preprocessor)
    print("✅ Preprocessor loaded successfully.")
except Exception as e:
    print(f"❌ Error loading preprocessor: {e}")
    preprocessor = None

# --- 3. Prepare Sample Data for Inference ---
if model and preprocessor:
    print("\nPreparing sample data for inference...")
    # This dictionary should match the exact feature names your model expects
    
    input_df = pd.DataFrame(example_input_data)
    print(f"Sample data columns: {list(input_df.columns)}")

    # --- 4. Preprocess and Predict ---
    try:
        # Preprocess the data using the loaded SKLearn object
        processed_data = preprocessor.transform(input_df)
        print("✅ Data successfully preprocessed.")

        # Convert processed NumPy array to PyTorch tensor
        input_tensor = torch.tensor(processed_data, dtype=torch.float32)

        # Make prediction
        with torch.no_grad():
            outputs = model(input_tensor)
            # Apply sigmoid/threshold for binary classification
            #probabilities = torch.sigmoid(outputs).cpu().numpy().flatten()
            predictions = (outputs > 0.5).float().cpu().numpy()

        print("\n--- INFERENCE RESULTS ---")
        print(f"Input features shape after processing: {processed_data.shape}")
        print(f"Final Prediction (0=No Churn, 1=Churn): {predictions}")
        
    except Exception as e:
        print(f"❌ Prediction failed during transform/forward pass: {e}")

Loading Model: churn_prediction_model:latest
✅ PyTorch Model loaded successfully.

Loading Preprocessor: churn_preprocessor:latest
✅ Preprocessor loaded successfully.

Preparing sample data for inference...
Sample data columns: ['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
✅ Data successfully preprocessed.

--- INFERENCE RESULTS ---
Input features shape after processing: (2, 13)
Final Prediction (0=No Churn, 1=Churn): [[1.]
 [0.]]
